In [214]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
import time
import re

### 파일 불러오기

In [187]:
# pd.read_csv('time1_title_set.txt')
with open("time1_title_set.txt", 'r', encoding='utf8') as f:
    book_list = [i.replace('\',\n','').replace(' \'','').replace('\'','') for i in f]
len(book_list)
book_list = book_list[::-1]

In [203]:
book_list[-1]

'1% 리더만 아는 유머의 법칙'

### 리스트 및 변수 생성 및 초기화

In [188]:
book_titles = list()
book_reviews = list() 
reviews_date = list()

book_abstract = list()
book_abstract_title = list()
book_publish_date = list()
just_review_list = list()

num=int(0)
list_num = int(0)

### 크롬 브라우저 생성하기

In [189]:
url='http://www.yes24.com/24/Category/Display/001001026'
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(3)

In [205]:
num = 100 

In [207]:
while True:
    
    num+=int(1)
    #크롤링을 완료한 경우 프로세스 탈출 및 종료(무한 반복문이라서 필요)
    if len(book_list[(num-1)*7:num*7])==0:
        print('process finished')
        break
        
        
    
    #책 검색결과 탭 7개로 출력을 위한 임시 책 7권 리스트 생성
    try:
        just_book_list = book_list[(num-1)*7:num*7]
#         just_book_list = ['쿠션']
        print( )
        print( )
        print(just_book_list)
    #만약 마지막 책 그룹 항목이 7권이 아닐 경우 예외처리
    except IndexError:
        just_book_list = book_list[(num-1)*7:]
    
    #임시 책 7권 리스트 바탕 7개의 검색결과 탭 생성    
    for i in just_book_list:
        tab = driver.window_handles[0]
        driver.switch_to.window(window_name=tab)
        search = driver.find_element_by_css_selector('#query')
        search.clear() #검색창 입력어 지우기
        search.send_keys(i) #새로운 검색어 입력
        click = driver.find_element_by_css_selector('#yesSForm > fieldset > span.schBtn > button')
        click.send_keys(Keys.CONTROL +"\n") #클릭해서 새 탭으로 열리기
    
    
    
    #책 상세페이지 진입을 위한 과정    
    #1단계: 탭전환 및 방해할만한 요소 제거
    while True:
        #탭 전환과정 - 매번 두번째 탭(index 기준1)을 클릭할 수 있도록 조치
        #만약 두번째 탭이 존재하지 않을 경우 임시 책 7권 리스트를 모두 순환한 것으로 인식하고, 위에서 다시 7권 리스트 생성
        try:
            tab = driver.window_handles[1]
            driver.implicitly_wait(3)
            driver.switch_to.window(window_name=tab)
            
            #도중 오류 발생 방지를 위해 yes24 팝업 광고 제거
            try:
                driver.find_element_by_css_selector('#divYes24SCMEvent > div.yPopBot > div.pTxt > label').click()
#                 time.sleep(2) #테스트
            except ElementNotInteractableException:
                pass

            
            
     #2단계: 실제 책 검색한 책이름과 검색결과 항목(책 제목)과 대조 하기
            #인기순(default값) 기준 첫번째 항목 확인 및 대조
            title = driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
            if title in just_book_list:
                u=1
                
            #아닐 경우, 정밀한 검색을 위해 나열 기준 '정확도'로 변경 및 상위 리스트 10권 제목 확인
            if title not in just_book_list:
                driver.find_element_by_css_selector('#formTest > ul > li:nth-child(2) > a').send_keys(Keys.ENTER)
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
                u=1
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]/p[1]/a').text
                u=3
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[2]/p[1]/a').text
                u=5
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]/p[1]/a').text
                u=7
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[9]/td[2]/p[1]/a').text
                u=9
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[11]/td[2]/p[1]/a').text
                u=11
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td[2]/p[1]/a').text
                u=13
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[15]/td[2]/p[1]/a').text
                u=15
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[17]/td[2]/p[1]/a').text
                u=17
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[19]/td[2]/p[1]/a').text
                u=18
            #책 제목이 존재할 경우 책 상세페이지로 입장
            #만약 동일한 제목이 없을 경우, 3단계 과정 진행시 오류가 나면서 에러처리로 탭이 닫힘(예외처리 참조)
            if title in just_book_list:
                driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[{u}]/td[2]/p[1]/a').click()
                #진행 상황 확인할 수 있도록 하기위해 책 이름 및 번째 책 항목인지 출력
                list_num +=1
                print()
                print(f'[{list_num}] {title}')
         
    
    #3단계: 책 초록 긁어오기
            try:
                publish_date = driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date').text
            except NoSuchElementException:
                publish_date = None
                
            try:
                abstract1 = (driver.find_element_by_css_selector('#infoset_introduce > div.infoSetCont_wrap > table')).text
            except NoSuchElementException:
                try:
                    abstract1 = (driver.find_element_by_css_selector('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt')).text
                except NoSuchElementException:
                    pass
            try:
                abstract2 = (driver.find_element_by_css_selector('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')).text
            except NoSuchElementException:
                try:
                    abstract2 = '' #단원목록이 없는 경우에 해당
                except NoSuchElementException:
                    pass
            
            book_publish_date.append(publish_date)
            book_abstract_title.append(title)
            book_abstract.append((abstract1 + abstract2).replace('\n',' ') )
#             print(title, abstract1 + abstract2[0:10])
#             print()
                
    #4단계: 책 상세페이지에서 들어가 크롤링
            time.sleep(2)
            review_num = 0
            try:
                title = driver.find_element_by_xpath('/html/body/div/div[4]/div[2]/div[1]/div/h2')
                title = title.text
                
#                 #진행 상황 확인할 수 있도록 하기위해 책 이름 및 번째 책 항목인지 출력
#                 list_num +=1
#                 print(f'[{list_num}] {title}')
                    
                for k in list(range(2)):
                    for j in list(range(10)):

                        #리뷰 단일 페이지 크롤링
                        #모든 항목(최대 5개 리뷰) 리뷰 더보기 클릭하기
                        for i in list(range(5)):
                            
                            element = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+str(i+2)+']/div[2]/a/div/span')
                            driver.execute_script("arguments[0].click();", element)
                            time.sleep(0.5) #혹시나 인터넷 속도가 감당 못할 경우 시간 늘리기
            
                            
                            
                        #페이지내 각 리뷰별 리뷰,날짜 텍스트 수집
                        
#                         yes = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div[2]/div[3]/div[2]').text
                        
#                         print()
#                         print()
#                         print(yes[:10])
#                         print(just_review_list)
                        
                        for i in list(range(5)):
                            review_num += 1
                            print(review_num, end=' ')
                            yes = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+ str(i+2) +']/div[3]/div[2]')
                            date = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div[2]/div[1]/div/em[4]')

    #                                 print(date.text)
#                                         print(yes.text)
    #                                 print()
                            book_titles.append(title)
                            book_reviews.append(yes.text.replace('\n',' ')) 
                            reviews_date.append(date.text)
                            just_review_list.append(yes.text[:10])
                        
#                         print('-------------------------------------페이지 '+ str(k) + str(j+1) + ' 끝---------------------------------------')
                            
                        #다음 리뷰 페이지로 진입
                        yes = driver.find_element_by_css_selector('#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child(' + str(j+4) +')')                    
                        yes.click()
                        time.sleep(1)
                driver.close()
            except NoSuchElementException:                
                driver.close()
                pass
            except ElementNotInteractableException:
                driver.close()
                pass
            except ElementClickInterceptedException :
                driver.close()
                pass
        except NoSuchElementException:                
                driver.close()
        except ElementNotInteractableException:
            driver.close()
#         except ElementClickInterceptedException :
#             driver.close()
#             break
        except IndexError:
#             print('start new books process')
            break
        
    
#  #infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child(10)       



['가슴 뛰는 삶', '가까운 사람들과 편하게 지내는 법', 'Think Innovation 씽크 이노베이션', 'The TOOLS 툴스', 'The Secret 시크릿', 'The Power 파워', 'The Harmony 조화로운 인생']

[684] 가슴 뛰는 삶
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
[685] 가까운 사람들과 편하게 지내는 법
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
[686] Think Innovation 씽크 이노베이션
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 
[687] The TOOLS 툴스
1 2 3 4 5 6 7 8 9 10 
[688] The Secret 시크릿
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 9

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 
[742] 1인 혁명가가 되라
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
[743] 1분 몰입

[744] 1분 경영수업
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 
[745] 1만 시간의 법칙
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

['1년만 미쳐라', '18시간 몰입의 법칙', '18분', '14살 세상 끝의 좌절, 23살 세상 속으로의 도전', '10미터만 더 뛰어봐!', '10년 후, 나를 디자인한다', '100세 현역 의사의 스트레스 내려놓기 연습']

[746] 1년만 미쳐라
1 2 3 4 5 6 7 8 9 10 
[747] 18시간 몰입의 법칙
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 
[748] 18분
1 2 3 4 5 
[749] 14살 세상 끝의 좌절, 23살 세상 속으로의 도전
1 2 3 4 5 
[750] 10미터만 더 뛰어봐!
1 2 3 4 5 
[751] 10년 후, 나를 디자인한다

[752] 100세 현역 의사의 스트레스 내려놓기 연습
1 2 3 4 5 

['100달러로 세상에 뛰어들어라', '10 10 10 텐

In [208]:
reviews = pd.DataFrame(book_reviews)
titles = pd.DataFrame(book_titles)
dates = pd.DataFrame(reviews_date)
final_data= pd.concat([titles, reviews, dates],axis=1)
final_data = final_data.drop_duplicates(keep='first')
final_data

,0,0,0
0,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,나에게는 주말에 읽고 싶은 책 읽는 시간도 일상을 향유하는 쉼의 시간이다. 의지를 ...,2013-04-14
1,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,삶을 여행처럼 가볍고 즐겁게 살 수는 없을까? 참으로 기발한 접근이다. 은퇴후 ...,2013-04-14
2,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,여행 만큼 가슴 설레이는 일이 또 있을까? 나도 결혼하기 전에는 여행 정말 좋아했다...,2013-04-14
3,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,'여행자적 라이프스타일' 이야기. 여행 콘텐츠 사업가이자 관광학과 교수인 장준수는 ...,2013-04-14
4,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,"나는 여행을 좋아한다. 풍광이 아름다운 곳도 좋아하지만, 역사적으로 의미있는 곳들을...",2013-04-14
...,...,...,...
24350,1% 행운,사람이 살면서 3번의 기회가 온다고 한다. 하지만 대부분의 사람들은 그 기회조차 ...,2008-02-01
24351,1% 행운,가장 최근에 출간된 또 하나의 자기계발 서적 '내 인생에서 놓쳐선 안 될 1%행운'...,2008-02-01
24352,1% 행운,<마음을 열어주는 101가지 이야기> 시리즈와 <영혼을 위한 닭고기 수프> 시리...,2008-02-01
24353,1% 행운,요즘 내 인생의 미래에 대해서 곧잘 생각해본다. 20대후반.. 이제 곧 30대에 접...,2008-02-01


In [210]:
final_data.to_csv('time1 탑100 리뷰.csv', encoding='utf8')

In [109]:
book_abstract_title

,0
0,1% 리더만 아는 유머의 법칙
1,1% 행운
2,10 10 10 텐 텐 텐 인생이 달라지는 선택의 법칙
3,100달러로 세상에 뛰어들어라
4,100세 현역 의사의 스트레스 내려놓기 연습
5,"10년 후, 나를 디자인한다"
6,10미터만 더 뛰어봐!
7,"14살 세상 끝의 좌절, 23살 세상 속으로의 도전"
8,18분
9,18시간 몰입의 법칙


In [211]:
book_abstract_title = pd.DataFrame(book_abstract_title)
book_abstract = pd.DataFrame(book_abstract)
final_data= pd.concat([book_abstract_title, book_abstract],axis=1)
final_data = final_data.drop_duplicates(keep='first')
final_data

,0,0
0,힘들지 않은 인생 없고 즐겁지 않은 여행 없다,'행복한 라이프스타일을 가지고 있는 사람이 행복한 인생을 사는 사람'이라고 정의 내...
1,MAFIA HIT 마피아 히트,'행복한 라이프스타일을 가지고 있는 사람이 행복한 인생을 사는 사람'이라고 정의 내...
2,히든 커뮤니케이션,"커뮤니케이션의 9할은 ‘심리전’이다. 잘 다듬어진 한 번의 손짓, 표정, 질문은 논..."
3,흥하는 말씨 망하는 말투,말을 바꾸면 인격이 변한다. 말을 바꾸면 운명도 변한다. 말은 그 사람의 운명을 운...
4,흔들리지 않고 피어나는 마흔은 없다,중년의 길목에 들어선 사람들은 온갖 고생을 다해왔지만 아무도 자기 마음을 몰라준다며...
...,...,...
772,100세 현역 의사의 스트레스 내려놓기 연습,이 책의 저자 히노하라 시게아키 박사(100). 1911년 10월4일생인 그는 얼마...
773,100달러로 세상에 뛰어들어라,"단돈 100달러로 좁은 사무실에서 벗어나 전 세계를 무대로 펼친, 당신만의 꿈을 위..."
774,10 10 10 텐 텐 텐 인생이 달라지는 선택의 법칙,무수한 선택 앞에서 망설이거나 중요한 결정을 앞두고 딜레마에 빠진 모든 현대인들에게...
775,1% 행운,감동의 눈물과 웃음으로 용기와 힘을 북돋우는 따뜻한 성공서 ‘영혼을 위한 닭고기수프...


In [212]:
final_data.to_csv('time1 탑100 초록.csv', encoding='utf8')